In [1]:
import numpy as np
from __future__ import division

filename = 'glove.6B.50d.txt' 
# (glove data set from: https://nlp.stanford.edu/projects/glove/)


def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('GloVe Loaded.')
    file.close()
    return vocab,embd

# Pre-trained GloVe embedding
vocab,embd = loadGloVe(filename)

embedding = np.asarray(embd)
embedding = embedding.astype(np.float32)

word_vec_dim = len(embd[0]) # word_vec_dim = dimension of each word vectors

GloVe Loaded.


In [2]:
import csv
import nltk as nlp
from nltk import word_tokenize
import string

summaries = []
texts = []

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    return filter(lambda x: x in printable, text) #filter funny characters, if any. 
    

with open('Reviews.csv', 'rb') as csvfile: #Data from https://www.kaggle.com/snap/amazon-fine-food-reviews
    Reviews = csv.DictReader(csvfile)
    for row in Reviews:
        clean_text = clean(row['Text'])
        clean_summary = clean(row['Summary'])
        summaries.append(word_tokenize(clean_summary))
        texts.append(word_tokenize(clean_text))


In [3]:
import random

index = random.randint(0,len(texts)-1)

print "SAMPLE CLEANED & TOKENIZED TEXT: \n\n"+str(texts[index])
print "\nSAMPLE CLEANED & TOKENIZED SUMMARY: \n\n"+str(summaries[index])

SAMPLE CLEANED & TOKENIZED TEXT: 

['my', 'kitty', 'is', '14', 'years', 'old', 'and', 'she', 'started', 'throwing', 'up', 'alot', 'this', 'spring', '.', 'i', 'switched', 'to', 'natural', 'balance', 'and', 'it', 'was', 'good', 'for', 'a', 'month', 'and', 'then', 'she', 'started', 'again', '.', 'so', 'i', 'tried', 'the', 'longevity', 'formula', 'and', 'so', 'far', 'have', 'had', 'great', 'results', '.', 'first', 'of', 'all', ',', 'she', 'seemed', 'to', 'like', 'the', 'taste', 'and', 'the', 'transition', 'to', 'a', 'new', 'food', 'was', 'a', 'breeze', '.', 'she', 'has', 'been', 'on', 'this', 'formula', 'for', 'a', 'couple', 'of', 'months', 'and', 'has', 'totally', 'quit', 'vomiting', '(', 'except', 'when', 'she', 'eats', 'grass', 'of', 'course', ')', ';', 'she', 'had', 'lost', 'some', 'weight', 'and', 'seems', 'more', 'fit', 'and', 'trim', '.', 'i', 'am', 'sooo', 'glad', 'and', 'highly', 'recommend', 'this', 'formula', '.', 'i', 'will', 'see', 'how', 'she', 'does', 'over', 'the', 'winter'

In [4]:
def np_nearest_neighbour(x):
    #returns array in embedding that's most similar (in terms of cosine similarity) to x
        
    xdoty = np.multiply(embedding,x)
    xdoty = np.sum(xdoty,1)
    xlen = np.square(x)
    xlen = np.sum(xlen,0)
    xlen = np.sqrt(xlen)
    ylen = np.square(embedding)
    ylen = np.sum(ylen,1)
    ylen = np.sqrt(ylen)
    xlenylen = np.multiply(xlen,ylen)
    cosine_similarities = np.divide(xdoty,xlenylen)

    return embedding[np.argmax(cosine_similarities)]
    


def word2vec(word):  # converts a given word into its vector representation
    if word in vocab:
        return embedding[vocab.index(word)]
    else:
        return embedding[vocab.index('unk')]

def vec2word(vec):   # converts a given vector representation into the represented word 
    for x in xrange(0, len(embedding)):
            if np.array_equal(embedding[x],np.asarray(vec)):
                return vocab[x]
    return vec2word(np_nearest_neighbour(np.asarray(vec)))


In [5]:
word = "unk"
print "Vector representation of '"+str(word)+"':\n"
print word2vec(word)

Vector representation of 'unk':

[ -7.91490018e-01   8.66169989e-01   1.19980000e-01   9.22870007e-04
   2.77599990e-01  -4.91849989e-01   5.01950026e-01   6.07919996e-04
  -2.58450001e-01   1.78650007e-01   2.53500015e-01   7.65720010e-01
   5.06640017e-01   4.02500004e-01  -2.13879999e-03  -2.83969998e-01
  -5.03239989e-01   3.04490000e-01   5.17790020e-01   1.50899999e-02
  -3.50309998e-01  -1.12779999e+00   3.32529992e-01  -3.52499992e-01
   4.13260013e-02   1.08630002e+00   3.39099988e-02   3.35640013e-01
   4.97449994e-01  -7.01309964e-02  -1.21920002e+00  -4.85119998e-01
  -3.85119990e-02  -1.35539994e-01  -1.63800001e-01   5.23209989e-01
  -3.13180000e-01  -1.65500000e-01   1.19089998e-01  -1.51150003e-01
  -1.56210005e-01  -6.26550019e-01  -6.23359978e-01  -4.21499997e-01
   4.18729991e-01  -9.24719989e-01   1.10490000e+00  -2.99959987e-01
  -6.30029989e-03   3.95399988e-01]


In [6]:
#REDUCE DATA (FOR SPEEDING UP THE NEXT STEPS)

MAXIMUM_DATA_NUM = 50000

texts = texts[0:MAXIMUM_DATA_NUM]
summaries = summaries[0:MAXIMUM_DATA_NUM]

In [8]:
vocab_limit = []
embd_limit = []

i=0
for text in texts:
    for word in text:
        if word not in vocab_limit:
            if word in vocab:
                vocab_limit.append(word)
                embd_limit.append(word2vec(word))

In [9]:
for summary in summaries:
    for word in summary:
        if word not in vocab_limit:
            if word in vocab:
                vocab_limit.append(word)
                embd_limit.append(word2vec(word))


In [10]:
if 'eos' not in vocab_limit:
    vocab_limit.append('eos')
    embd_limit.append(word2vec('eos'))
if 'unk' not in vocab_limit:
    vocab_limit.append('unk')
    embd_limit.append(word2vec('unk'))

null_vector = np.zeros([word_vec_dim])

vocab_limit.append('<PAD>')
embd_limit.append(null_vector)    

In [11]:
vec_summaries = []

for summary in summaries:
    
    vec_summary = []
    
    for word in summary:
        vec_summary.append(word2vec(word))
            
    vec_summary.append(word2vec('eos'))
    
    vec_summary = np.asarray(vec_summary)
    vec_summary = vec_summary.astype(np.float32)
    
    vec_summaries.append(vec_summary)

In [12]:
vec_texts = []

for text in texts:
    
    vec_text = []
    
    for word in text:
        vec_text.append(word2vec(word))
    
    vec_text = np.asarray(vec_text)
    vec_text = vec_text.astype(np.float32)
    
    vec_texts.append(vec_text)    

In [13]:
#Saving processed data in another file.

import pickle
with open('vocab_limit', 'wb') as fp:
    pickle.dump(vocab_limit, fp)
with open('embd_limit', 'wb') as fp:
    pickle.dump(embd_limit, fp)
with open('vec_summaries', 'wb') as fp:
    pickle.dump(vec_summaries, fp)
with open('vec_texts', 'wb') as fp:
    pickle.dump(vec_texts, fp)
